In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion();
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import utils_optimizers as uopt
import os
%matplotlib inline
np_complex = np.vectorize(complex)

In [ ]:
diag_dir = "../Data/test_run"

sys_params = tdg.define_system_params(diag_dir)

In [ ]:
def fitness_function(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = dataset["rms"][:,:]
    avg_flux = dataset["avg_flux"][:,:]
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux) * norm_factor
    return maxi_func

In [ ]:
dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
opt_params = uopt.define_optimizer_parameters(diag_dir, 0, 0, dataset_params, facility_spec, sys_params)

fitness_temporal = fitness_function(dataset, opt_params)
fitness_overall = uopt.fitness_function(dataset, opt_params)

labels = [None] * dataset_params["num_examples"]
for iconfig in range(dataset_params["num_examples"]):
    labels[iconfig] = r"SGO={:.2f} and $R_b$={:.2f} $\mu m$".format(deck_gen_params["beamspot_order"][iconfig,0], deck_gen_params["beamspot_major_radius"][iconfig,0])

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)

cut = 10
ax1.semilogy(dataset_params["plasma_profile_times"], fitness_temporal[:cut].T, label=labels[:cut])
ax1.semilogy(dataset_params["plasma_profile_times"], fitness_temporal[cut:].T, ":", label=labels[cut:])
ax2.plot(fitness_overall.T, label=labels)

xlims = [0.,30.0]
ylims = [0.01,10.0]

ax1.set_xlim(xlims)
ax1.set_ylim(ylims)
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("Fitness")
ax1.legend(loc="best")

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)

cut = 10
ax1.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][:cut].T * 100, label=labels[:cut])
ax1.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][cut:].T * 100, ":", label=labels[cut:])
ax2.plot(dataset_params["plasma_profile_times"][1:],dataset["avg_flux"][:cut,1:].T, label=labels[:cut])
ax2.plot(dataset_params["plasma_profile_times"][1:],dataset["avg_flux"][cut:,1:].T, ":", label=labels[cut:])

xlims = [0.,30.0]
ylims = [0.,100.0]

ax1.set_xlim(xlims)
#ax1.set_ylim(ylims)
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("RMS ablation pressure (%)")
ax1.legend(loc="best")
#fig1.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_intensity.png', bbox_inches='tight')

ax2.set_xlim(xlims)
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Ablation pressure (Mbars)")
ax2.legend(loc="best")
#fig2.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_ablation_pressure_estimate.png', bbox_inches='tight')